#### Code for  adapted from https://github.com/lokinou/p3k_offline_analysis

commit e273e9e42380d6c117feec1ef7ca00851b09092e (06.08.21)

In [1]:
# uncomment the line wit qt below to obtain separate plots to save
%matplotlib inline
#%matplotlib qt
#matplotlib.use('Qt5Agg')

### Adaptaions:
- microvolt/volt scaling in function
- channel names are adjusted for the sessions with changed postions 
    since they were still plugged in the same positions to the pre-Amp they need to be called by the wrong names when looking for the EEG signal in the loadBCI2k function

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import os
import glob
import mne
import pandas as pd
import itertools
import re
import matplotlib
import pickle
from pathlib import Path
# LDA
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from BCI2kReader import BCI2kReader as b2k
from BCI2kReader import FileReader as f2k


In [20]:
raws = []

def extract_annotations(filename, verbose=False):
    display_preprocessing_plots = False
    file= b2k.BCI2kReader(filename)

    if verbose:
        print(file.states)
    target_states = np.squeeze(file.states['StimulusType'])
    stimulus_codes = np.squeeze(file.states['StimulusCode'])
    if 'StimulusBegin' in file.states.keys():
        stimulus_begin = np.squeeze(file.states['StimulusBegin'])
    else:
        stimulus_begin = np.squeeze(file.states['Flashing'])
    
    phase = np.squeeze(file.states['PhaseInSequence'])

    fs = file.samplingrate

    idx_targets = np.where(target_states)[0]
    idx_codes = np.where(stimulus_codes>0)[0]
    idx_begin = np.where(stimulus_begin>0)[0]


    # In BCI2000 states are maintained over different samples, we search here the differences of when the codes are > 0
    groups = np.split(idx_codes, np.where(np.diff(idx_codes) != 1)[0]+1)
    # we take the first sample where a difference can be found
    code_change_idx = np.array([g[0] for g in groups])
    #[idx_codes[idx] for idx in code_change_idx]
    print('nb stimuli={}'.format(len(code_change_idx)))

    # we intersect the target index list with the code change to find the onset of targets and non-targets
    target_idx=np.intersect1d(code_change_idx,idx_targets)
    print('nb targets={}'.format(len(target_idx)))
    non_target_idx= np.setdiff1d(code_change_idx,idx_targets)

    # Translating into MNE Annotations 
    # define the annotations from the recovered stimuli (in seconds)
    sample_lengh = 1/fs
    onsets = code_change_idx * sample_lengh
    onsets = np.repeat(onsets, 2)  # repeat onsets
    # define the descriptio
    description_targets = np.zeros(code_change_idx.shape, dtype=np.uint)
    # index of targets in the list of stimuli onsets
    description_targets[np.searchsorted(code_change_idx, target_idx)] = 1
    description_codes = stimulus_codes[code_change_idx] + stimulus_padding  # start codes at 100 because 0 and 1 are used for target and nontarget
    # merge code and target decriptions
    description = np.zeros(description_targets.shape[0]*2, dtype=np.uint)
    description[np.arange(description_targets.shape[0]*2, step=2)] = description_codes
    description[np.arange(start=1, stop=(description_targets.shape[0]*2)+1, step=2)] = description_targets

    if display_preprocessing_plots:
        fig = plt.figure()
        plt.plot(description[:100])
        fig.suptitle('Targets(1) and non-targets(0) for 100 first stimuli')

    if display_preprocessing_plots:
        fig = plt.figure()
        plt.plot(phase == 1)
        fig.suptitle('Trial begin')

    # extract trial begin markers  #  this method does not work since some stimuli are declared before phase==1
    # let's think baclwards use the end markers instead
    new_phase_continuous = np.where(phase == 1)[0]
    groups = np.split(new_phase_continuous, np.where(np.diff(new_phase_continuous) != 1)[0]+1)
    new_trial_idx = np.array([g[0] for g in groups]) if len(groups) > 1 else None
    
    
    # extract trial end markers
    new_phase_continuous = np.where(phase == 3)[0]
    groups = np.split(new_phase_continuous, np.where(np.diff(new_phase_continuous) != 1)[0]+1)
    end_of_trial_idx = np.array([g[-1] for g in groups]) # take the last index to integrate all post sequence duration
    
    # deduce trial begin markers  # 
    #new_trial_idx = np.zeros(end_of_trial_idx.size)
    #new_trial_idx[1:] = end_of_trial_idx[1:]+1

    print(new_trial_idx)
    print(end_of_trial_idx)
    
    
    if new_trial_idx is None:
        print('WARNING: markers for begin trial (PhaseInSequence=1) missing (in old brain painting dev versions)!!!, using end of trial instead')
        new_trial_idx = [0]
        new_trial_idx.extend(end_of_trial_idx[0:-1])  # deduce the bounds from end of trial
        new_trial_idx = np.array(new_trial_idx)  # convert to numpy
        print(new_trial_idx)
    
    if new_trial_idx.shape[0] > end_of_trial_idx.shape[0]:
        print('WARNING: no end of trial for the last trial (interrupted recording?), it will be ignored for offline accuracy calculation')
        inter_trial_duration = end_of_trial_idx[0:len(new_trial_idx)] - new_trial_idx
    else:
        inter_trial_duration = end_of_trial_idx - new_trial_idx
        
    inter_trial_duration = inter_trial_duration * sample_lengh  # express in seconds


    print("Extracted {} trials".format(len(new_trial_idx)))

    # set a non-zero duration for stimuli (or MNE ignores them)
    duration = np.ones(onsets.shape) * sample_lengh


    # merge phase in sequence events with stimuli onsets
    onsets_phase = new_trial_idx * sample_lengh
    onsets = np.concatenate((onsets, onsets_phase))
    
    duration = np.concatenate((duration, inter_trial_duration))
    description = np.concatenate((description, np.ones(new_trial_idx.shape) * 10))  # concatenate trials markers=10
    srt = np.argsort(onsets) # sort according to their timing
    onsets=onsets[srt]
    duration = duration[srt]
    description = description[srt].astype(np.uint8)
    inter_trial_duration
    annotations = mne.Annotations(onset=onsets, duration=duration, description=description)

    file.flush()
    return annotations

def load_bci2k(filename_list, verbose=False):
    """
    return MNE raw, number of rows in the matrix
    """
    raws = []
    for fn in filename_list:
        cname = None
        with b2k.BCI2kReader(fn) as file:
            
            # Extract signals and states
            print('Reading {}'.format(fn))
            eeg_data = file.signals
            states = file.states
            fs = file.samplingrate
            nb_chan = eeg_data.shape[0]
            #file.purge()

            # Extract channel names
            reader = f2k.bcistream(fn)
            if verbose:
                print(reader.params)
            # actualize the parameters by including the defined channel names
            if len(reader.params['ChannelNames']):
                if cname != reader.params['ChannelNames']:
                    cname = reader.params['ChannelNames']
                    print('Actualized channel names to {}'.format(cname))

            if cname is None:
                cname = [str(ch_n) for ch_n in list(range(nb_chan))]
                
            # extract the number of rows
            nb_stim_rows = np.uint8(reader.params['NumMatrixRows'][0])
            nb_stim_cols = np.uint8(reader.params['NumMatrixColumns'][0])
            nb_seq = np.uint8(reader.params['NumberOfSequences'])

            # convert states into annotations
            info = mne.create_info(cname, fs, ch_types='eeg', verbose=None)
            raw_array = mne.io.RawArray(eeg_data, info)
            # Manually force the filename or mne complains
            raw_array._filenames = [os.path.basename(fn)]
            
            # If the variance of the data is >1, it means the data is expressed in microvolts
            # Since MNE uses Volt as a default value, we rescale microvolts to volt
            if np.var(raw_array._data)>1:
                raw_array._data = raw_array._data * 1.e-6
                print('Rescaled signal to Volt (mean variance={})'.format(np.var(raw_array._data)))
            
            annotations = extract_annotations(fn, verbose= False)
            raw_array.set_annotations(annotations)
            raws.append(raw_array)
    return raws, (nb_stim_rows, nb_stim_cols, nb_seq)

#fn = ["./data_sample/bci2000\Heide_einsteinBP_calibration4S001R01.dat"]



In [23]:
mne.set_log_level("CRITICAL")

In [24]:
stimulus_padding  = 100
old_cnames = ["Fz","Fc1","Fc2","C3","Cz","C4","P3","Pz","P4","O1","Oz","O2"]
new_cnames = ["Fz","Fc1","Fc2","C3","C1","Cz","C2","C4","Cp1","Cpz","Cp2","Pz"] # changed on 12.12.20 (sess021)
CNAMES = dict(zip(old_cnames,new_cnames))
os.chdir(r"G:\Meine Ablage\PhD\P300\Data\dat_files")
session_list = list(range(1,31)) #.append("all")
pickle_path = r"G:\Meine Ablage\PhD\P300\Data\mne_raw" 
for sess in range(1,31):
    for mode in ["Calib","Copy"]:
        fnames = [i for i in glob.glob("WT-tac"+mode+f"{sess:03}"+"\WT-tac"+mode+"S*"+f"{sess:03}"+"R*.dat")] #WT-tac___S___R____.dat
        if len(fnames)!=3: # check if all trials are there, if not exclude session
            print("ATTENTION__________________________________ Something wrong with session {}".format(sess))
        signal_with_matrix_params = load_bci2k(fnames,verbose=False)
        #print("Using from parameters n_rows={}, n_cols={}, n_seq={}".format(nb_stimlus_rows, nb_stimulus_cols, nb_seq))
        extension = mode+"_S"+str(sess).zfill(3)+"_4tac.pickle"
        with open(os.path.join(pickle_path,extension), "wb") as file:
            pickle.dump(signal_with_matrix_params, file)

Reading WT-tacCalib001\WT-tacCalibS001R01.dat


Actualized channel names to ['Fz', 'Fc1', 'Fc2', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4', 'O1', 'Oz', 'O2']
Rescaled signal to Volt (mean variance=2.5845430526843326e-10)


nb stimuli=320
nb targets=80
[ 1024 14784 28544 42304 56064 69824 83584 97344]
[ 14783  28543  42303  56063  69823  83583  97343 111103]
Extracted 8 trials
Reading WT-tacCalib001\WT-tacCalibS001R05.dat


Actualized channel names to ['Fz', 'Fc1', 'Fc2', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4', 'O1', 'Oz', 'O2']
Rescaled signal to Volt (mean variance=2.8952849062533285e-10)


nb stimuli=320
nb targets=80
[ 1024 14784 28544 42304 56064 69824 83584 97344]
[ 14783  28543  42303  56063  69823  83583  97343 111103]
Extracted 8 trials
Reading WT-tacCalib001\WT-tacCalibS001R06.dat


...

Actualized channel names to ['Fz', 'Fc1', 'Fc2', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4', 'O1', 'Oz', 'O2']
Rescaled signal to Volt (mean variance=1.0178784669897086e-09)


nb stimuli=256
nb targets=64
[ 1024 12800 24576 36352 48128 59904 71680 83456]
[12799 24575 36351 48127 59903 71679 83455 95231]
Extracted 8 trials
Reading WT-tacCopy030\WT-tacCopyS030R02.dat


Actualized channel names to ['Fz', 'Fc1', 'Fc2', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4', 'O1', 'Oz', 'O2']
Rescaled signal to Volt (mean variance=7.60117377093259e-10)


nb stimuli=256
nb targets=64
[ 1024 12800 24576 36352 48128 59904 71680 83456]
[12799 24575 36351 48127 59903 71679 83455 95231]
Extracted 8 trials
Reading WT-tacCopy030\WT-tacCopyS030R03.dat


Actualized channel names to ['Fz', 'Fc1', 'Fc2', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4', 'O1', 'Oz', 'O2']
Rescaled signal to Volt (mean variance=6.389175140443445e-10)
nb stimuli=256
nb targets=64
[ 1024 12800 24576 36352 48128 59904 71680 83456]
[12799 24575 36351 48127 59903 71679 83455 95231]
Extracted 8 trials


In [25]:
signal_with_matrix_params[0][2].describe()

<RawArray | WT-tacCopyS030R03.dat, 12 x 96256 (188.0 s), ~8.8 MB, data loaded>
ch  name  type  unit       min        Q1    median        Q3       max
 0  Fz    EEG   µV     -173.28    -20.53      0.36     22.97    315.81
 1  Fc1   EEG   µV     -138.47    -14.42      0.46     15.87    219.78
 2  Fc2   EEG   µV     -114.68    -13.43      0.20     14.52    196.91
 3  C3    EEG   µV     -116.85    -12.34      0.13     11.58    181.47
 4  Cz    EEG   µV      -96.79    -10.66      0.30     11.48    157.01
 5  C4    EEG   µV     -151.04    -10.69      0.26     10.92    142.07
 6  P3    EEG   µV      -88.67     -9.21      0.03      8.48    130.55
 7  Pz    EEG   µV      -83.56     -9.17      0.30      9.31    122.56
 8  P4    EEG   µV      -75.01     -8.71      0.16      8.58    113.75
 9  O1    EEG   µV     -142.13     -7.99     -0.20      7.33    102.99
10  Oz    EEG   µV      -72.40     -7.36      0.16      7.16     96.46
11  O2    EEG   µV     -113.59     -7.19      0.03      6.90     91.6